# Explore Weather Data from BSEC Weather Stations

Welcome! This notebook will demonstrate how to use the MSD-LIVE data exploration service to visualize, transform, and download a subset of data collected from weather stations around the Baltimore area.

You can find the original dataset record at [https://doi.org/10.57931/2476281](https://doi.org/10.57931/2476281).


### Install Dependencies

First, we need to install a few libraries that we will use to read and transform the data.

To do so, we'll use the `!` directive to access the shell and then use `pip` to install our dependencies as specified in the accompanying [`requirements.txt`](requirements.txt).

To prevent the log messages from overflowing this notebook, we'll silence the output by directing informational messages into oblivion.


In [ ]:

!pip install -r requirements.txt > /dev/null



If you find a need for other libraries, you can use a similar technique to install them directly, for instance by running `!pip install matplotlib`


### Import Dependencies

Now we can import the libraries we want to use explore the dataset.


In [ ]:
import geopandas as gpd
import pandas as pd
import contextily as cx

from glob import glob
import os


### Find the Data

In this MSD-LIVE Jupyter environment, the dataset location is stored in the `DATA_DIR` environment variable.

We can again use the `!` directive to list the contents of this directory using shell commands, and we can use Python's `os.getenv()` method to get the path in code.

The data is also mounted to the `/data` directory in this Jupyter environment, so you can use the File Browser to see the available files and directory structure.


In [ ]:

!ls $DATA_DIR


### Explore the Data

Now let's use `pandas` and `geopandas` to read the CSV file of weather station locations.


In [ ]:

# Read the CSV using the DATA_DIR environment variable as the path
stations = pd.read_csv('Stations_Locations.csv')[[
    # Select a few interesting columns
    'Station ID', 'Site Name', 'Longitude', 'Latitude'
]]

# Convert the coordinates to a geometry column using geopandas
stations = gpd.GeoDataFrame(
    stations, geometry=gpd.points_from_xy(stations.Longitude, stations.Latitude)
).set_crs('epsg:4326')

# View the first few records
stations.head()



We can plot the stations locations on top of a basemap using `contextily` to get a sense for their spatial locations.


In [ ]:

# Reproject the geometries to Web Mercator, then plot on top of a muted basemap
ax = stations.to_crs('epsg:3857').plot(
    figsize=(7.2, 7.2), marker='o', markersize=10,
    color='red', edgecolor='black', linewidth=1,
)
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)
ax.set_axis_off()



### Subset the Data

Let's figure out the mean daily temperature at each station on your birthday!

Update your birthday below, then we'll read in all the CSV files for 2024 and select the observations on the chosen date.


In [ ]:

# NOTE that the dataset currently only goes through October, so you may need to fudge your birthday a little to see data...
birthday_month = 10
birthday_day   = 31


In [ ]:

# Read in each daily file, select the date, and add the observations to a new dataframe

daily_files = sorted(glob(f'{os.getenv("DATA_DIR")}/data/daily/2024/*.csv'))

birthday_data = []

for file in daily_files:
    
    df = pd.read_csv(file)
    
    # ensure the dates are datetime objects
    df['obsTimeUtc'] = pd.to_datetime(df.obsTimeUtc)
    
    # add observations on the chosen date to a list
    birthday_data.append(
        df[
            (df.obsTimeUtc.dt.month == birthday_month) &
            (df.obsTimeUtc.dt.day   == birthday_day)
        ]
    )

# combine all observations from all weather stations on the chosen date into a new dataframe
birthday_data = pd.concat(birthday_data, ignore_index=True)

# View the first few records
birthday_data.head()


<br/>


Let's create a histogram of all the temperatures to see the distribution of observations across weather stations:


In [ ]:

ax = birthday_data.tempAvg.hist();
ax.set_xlabel('Average Temperature °C');
ax.set_ylabel('Observations');
ax.set_title(f'Average Temperature Observations on 2024-{str(birthday_month).zfill(2)}-{str(birthday_day).zfill(2)} across all BSEC Weather Stations');


<br/>


### Saving the Data Subset

We can also save this new dataframe and download it to a local device for later use.

There are two ways to do this in the MSD-LIVE Jupyter environment.

* The easy way: for small files like this one, we can just write the file into the working directory and download it using the File Browser within Jupyter.
* For larger files (>100MB), we need to copy the file into our `scratch` directory and then use the MSD-LIVE CLI from our local device to download it.

We'll demonstrate the first option here and then show you how to copy the file to your `scratch`. For more information on downloading from the `scratch` folder, check out [How to download files from your scratch directory using our CLI](https://msdlive.org/help/resources/quick-guides/using-dataset-notebooks#scratch-download).




<br/>

In [ ]:

# Save the birthday subset dataframe to the current directory
birthday_data.to_csv('birthday_weather_station_data.csv', index=False)



You should now be able to see the data in the file explorer at [/notebooks/birthday_weather_station_data.csv](birthday_weather_station_data.csv). Right click this file in the File Browser and select "Download" to copy the file locally.

If the file was too big to download directly, you could first right click it and select "Copy to Scratch", and then follow the directions linked above to use the MSD-LIVE CLI to download from your `scratch` directory.


<br/>

Thanks for following along; you should now be empowered to explore the data for your own purposes!